# Product Team Sync → Google Doc

This notebook authenticates with the Google Docs API, parses the provided Markdown meeting notes, and programmatically creates a fully formatted Google Doc (headings, nested bullets, checkboxes, @mentions, and footer styling).

**Tip:** Run each cell in order, authenticate with your Google account when prompted, then rerun the final cell whenever you update the meeting notes.

In [1]:
!pip install --quiet google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import default

SCOPES = ["https://www.googleapis.com/auth/documents"]
DEFAULT_DOCUMENT_TITLE = "Product Team Sync - May 15, 2023"


def initialize_docs_service(scopes=None):
    """Authenticate in Colab and create a Google Docs API client."""
    scopes = scopes or SCOPES
    auth.authenticate_user()
    credentials, _ = default(scopes=scopes)
    return build("docs", "v1", credentials=credentials)


try:
    docs_service = initialize_docs_service()
    print("Google Docs API client initialized.")
except Exception as error:
    raise RuntimeError(f"Unable to initialize Google Docs API client: {error}") from error

In [ ]:
import re
from dataclasses import dataclass, field
from typing import List, Tuple

MEETING_NOTES_MD = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


@dataclass
class Block:
    text: str
    block_type: str
    level: int = 0
    assignee_spans: List[Tuple[int, int]] = field(default_factory=list)


CHECKBOX_PATTERN = re.compile(r"^(\s*)- \[ \] (.+)")
BULLET_PATTERN = re.compile(r"^(\s*)[*-] (.+)")
ASSIGNEE_PATTERN = re.compile(r"@\w+")


def level_from_indent(indent: str) -> int:
    """Translate leading spaces into a nesting level (2 spaces per level)."""
    return max(len(indent) // 2, 0)


def extract_assignee_spans(text: str) -> List[Tuple[int, int]]:
    """Return the index spans for @mentions so we can style them later."""
    return [(match.start(), match.end()) for match in ASSIGNEE_PATTERN.finditer(text)]


def parse_markdown(markdown_text: str) -> List[Block]:
    """Convert Markdown into Block objects for easier Google Docs formatting."""
    clean_text = markdown_text.strip()
    if not clean_text:
        return []

    blocks: List[Block] = []
    footer_mode = False
    for raw_line in markdown_text.splitlines():
        line = raw_line.rstrip()
        stripped = line.strip()

        if not stripped:
            continue
        if stripped == "---":
            footer_mode = True
            continue
        if footer_mode:
            blocks.append(Block(text=stripped, block_type="footer"))
            continue

        if stripped.startswith("### "):
            blocks.append(Block(text=stripped[4:].strip(), block_type="heading3"))
            continue
        if stripped.startswith("## "):
            blocks.append(Block(text=stripped[3:].strip(), block_type="heading2"))
            continue
        if stripped.startswith("# "):
            blocks.append(Block(text=stripped[2:].strip(), block_type="heading1"))
            continue

        checkbox_match = CHECKBOX_PATTERN.match(line)
        if checkbox_match:
            indent, task_text = checkbox_match.groups()
            text = task_text.strip()
            blocks.append(
                Block(
                    text=text,
                    block_type="checkbox",
                    level=level_from_indent(indent),
                    assignee_spans=extract_assignee_spans(text),
                )
            )
            continue

        bullet_match = BULLET_PATTERN.match(line)
        if bullet_match:
            indent, item_text = bullet_match.groups()
            text = item_text.strip()
            blocks.append(
                Block(
                    text=text,
                    block_type="bullet",
                    level=level_from_indent(indent),
                    assignee_spans=extract_assignee_spans(text),
                )
            )
            continue

        blocks.append(Block(text=stripped, block_type="paragraph", assignee_spans=extract_assignee_spans(stripped)))

    return blocks


def load_notes_from_path(path: str) -> str:
    """Optional helper if you upload a .md file to Colab and want to read it."""
    with open(path, "r", encoding="utf-8") as source:
        return source.read()


In [ ]:
def build_document_requests(blocks: List[Block]) -> List[dict]:
    """Translate parsed Blocks into Google Docs batchUpdate requests."""
    requests: List[dict] = []
    cursor = 1  # Start after the document start token.

    for block in blocks:
        text = f"{block.text}\n"
        requests.append({"insertText": {"location": {"index": cursor}, "text": text}})
        start_index = cursor
        end_index = cursor + len(text)
        cursor = end_index

        if block.block_type == "heading1":
            requests.append(
                {
                    "updateParagraphStyle": {
                        "range": {"startIndex": start_index, "endIndex": end_index},
                        "paragraphStyle": {"namedStyleType": "HEADING_1"},
                        "fields": "namedStyleType",
                    }
                }
            )
        elif block.block_type == "heading2":
            requests.append(
                {
                    "updateParagraphStyle": {
                        "range": {"startIndex": start_index, "endIndex": end_index},
                        "paragraphStyle": {"namedStyleType": "HEADING_2"},
                        "fields": "namedStyleType",
                    }
                }
            )
        elif block.block_type == "heading3":
            requests.append(
                {
                    "updateParagraphStyle": {
                        "range": {"startIndex": start_index, "endIndex": end_index},
                        "paragraphStyle": {"namedStyleType": "HEADING_3"},
                        "fields": "namedStyleType",
                    }
                }
            )
        elif block.block_type == "footer":
            requests.append(
                {
                    "updateTextStyle": {
                        "range": {"startIndex": start_index, "endIndex": end_index - 1},
                        "textStyle": {
                            "italic": True,
                            "foregroundColor": {"color": {"rgbColor": {"red": 0.3, "green": 0.3, "blue": 0.3}}},
                        },
                        "fields": "italic,foregroundColor",
                    }
                }
            )

        if block.block_type in {"bullet", "checkbox"}:
            preset = "BULLET_CHECKBOX" if block.block_type == "checkbox" else "BULLET_DISC_CIRCLE_SQUARE"
            requests.append(
                {
                    "createParagraphBullets": {
                        "range": {"startIndex": start_index, "endIndex": end_index},
                        "bulletPreset": preset,
                    }
                }
            )
            if block.level > 0:
                indent_points = 18 * block.level  # 18pt per level matches Docs defaults.
                requests.append(
                    {
                        "updateParagraphStyle": {
                            "range": {"startIndex": start_index, "endIndex": end_index},
                            "paragraphStyle": {
                                "indentFirstLine": {"magnitude": indent_points, "unit": "PT"},
                                "indentStart": {"magnitude": indent_points, "unit": "PT"},
                            },
                            "fields": "indentFirstLine,indentStart",
                        }
                    }
                )

        if block.assignee_spans:
            for span_start, span_end in block.assignee_spans:
                requests.append(
                    {
                        "updateTextStyle": {
                            "range": {
                                "startIndex": start_index + span_start,
                                "endIndex": start_index + span_end,
                            },
                            "textStyle": {
                                "bold": True,
                                "foregroundColor": {"color": {"rgbColor": {"red": 0.16, "green": 0.38, "blue": 0.78}}},
                            },
                            "fields": "bold,foregroundColor",
                        }
                    }
                )

    return requests


def create_document(service, title: str):
    """Create an empty Google Doc with the supplied title."""
    try:
        return service.documents().create(body={"title": title}).execute()
    except HttpError as error:
        raise RuntimeError(f"Failed to create Google Doc: {error}") from error


def populate_document(service, document_id: str, requests: List[dict]):
    """Send the batchUpdate request to populate and format the document."""
    if not requests:
        raise ValueError("No Google Docs requests were generated. Check the Markdown input.")

    try:
        service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()
    except HttpError as error:
        raise RuntimeError(f"Unable to populate Google Doc: {error}") from error


def build_google_doc(markdown_text: str, service, title: str = DEFAULT_DOCUMENT_TITLE):
    """Parse Markdown, create the document, and push the formatted content."""
    blocks = parse_markdown(markdown_text)
    requests = build_document_requests(blocks)
    document = create_document(service, title)
    populate_document(service, document["documentId"], requests)
    return document


In [ ]:
try:
    document_metadata = build_google_doc(MEETING_NOTES_MD, docs_service)
    doc_id = document_metadata.get("documentId")
    print("Google Doc created successfully!\n")
    print(f"Title: {document_metadata.get('title')}")
    print(f"Document ID: {doc_id}")
    print(f"URL: https://docs.google.com/document/d/{doc_id}/edit")
except Exception as error:
    print(f"Failed to build the document: {error}")
